In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display

In [2]:
notes = pd.read_csv('../../data/notes_cleaned.csv')

notes.head()

,note_id,id,note_type,note_seq,charttime,text,interval
0,18434803-RR-10,29998399,RR,10,2012-05-08 18:52:00,INDICATION: ___ rear ended motor cycle into s...,1
1,18434803-RR-11,29998399,RR,11,2012-05-08 19:09:00,EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICATI...,0
2,18434803-RR-12,29998399,RR,12,2012-05-08 19:09:00,EXAMINATION: CT C-SPINE W/O CONTRAST\n\nINDIC...,0
3,18434803-RR-13,29998399,RR,13,2012-05-08 19:10:00,EXAMINATION: CT torso\n\nINDICATION: ___ rea...,0
4,18434803-RR-14,29998399,RR,14,2012-05-08 19:12:00,EXAMINATION: SCROTAL U.S.\n\nINDICATION: ___...,0


In [3]:
dynamic = pd.read_csv('../../data/dynamic_cleaned.csv')

dynamic.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,28793466,4/12/29 3:35,16.0,21.0,12.0,8.8,106.0,0.6,154.0,139.0,3.8
1,26115624,9/7/50 0:22,12.0,22.0,9.0,7.9,111.0,0.6,97.0,141.0,3.6
2,28164589,3/11/59 1:11,12.0,34.0,58.0,8.3,103.0,2.1,89.0,146.0,3.4
3,20329785,5/12/34 19:32,11.0,38.0,31.0,10.1,88.0,0.8,95.0,133.0,4.0
4,24566943,6/25/55 15:45,16.0,22.0,45.0,8.7,106.0,2.2,173.0,140.0,4.0


In [4]:
static_binned = pd.read_csv('../../data/static_cleaned.csv')
static_binned = static_binned[static_binned['id'].isin(dynamic['id'])]

In [5]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = ohe.fit_transform(static_binned[['los_icu_binned']])
ohe_cols = ohe.get_feature_names_out(['los_icu_binned'])
ohe_los = pd.DataFrame(encoded, columns=ohe_cols)
static = pd.concat([static_binned, ohe_los], axis=1)

static.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,diabetes,hypertension,ihd,stroke,los_icu_binned,los_icu_binned_1 to 2 days,los_icu_binned_2 to 3 days,los_icu_binned_3 to 4 days,los_icu_binned_4 to 7 days,los_icu_binned_7+ days
0,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,...,1,1,0,0,3 to 4 days,0.0,0.0,1.0,0.0,0.0
1,29179088,4/10/80 23:21,4/17/80 14:00,4/12/80 17:41,4/16/80 13:38,3.83,0,1,WHITE,72.276455,...,0,1,1,0,4 to 7 days,0.0,0.0,0.0,1.0,0.0
2,27139242,4/25/33 5:47,5/13/33 7:15,4/25/33 6:37,4/27/33 16:01,2.39,0,1,BLACK/AFRICAN AMERICAN,73.312782,...,0,0,0,0,3 to 4 days,0.0,0.0,1.0,0.0,0.0
3,25918892,10/27/43 15:06,10/31/43 14:21,10/27/43 15:06,10/30/43 12:11,2.88,0,1,WHITE,91.820358,...,0,1,1,1,3 to 4 days,0.0,0.0,1.0,0.0,0.0
4,20778724,10/4/67 7:15,10/19/67 17:45,10/5/67 9:23,10/6/67 21:43,1.51,0,1,ASIAN - CHINESE,63.756003,...,0,1,0,1,1 to 2 days,1.0,0.0,0.0,0.0,0.0


In [6]:
static_dead = static[static['icu_death'] == 1]
static_alive = static[static['icu_death'] == 0]

display(static_dead.head())
display(static_alive.head())

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,diabetes,hypertension,ihd,stroke,los_icu_binned,los_icu_binned_1 to 2 days,los_icu_binned_2 to 3 days,los_icu_binned_3 to 4 days,los_icu_binned_4 to 7 days,los_icu_binned_7+ days
6,27780902,9/28/53 21:02,9/30/53 19:17,9/28/53 22:29,9/30/53 21:16,1.95,1,1,WHITE,47.741635,...,0,1,0,0,2 to 3 days,0.0,1.0,0.0,0.0,0.0
12,22987108,6/10/46 16:37,7/12/46 0:00,6/22/46 11:46,7/13/46 0:27,20.53,1,1,WHITE,69.439961,...,0,1,1,0,7+ days,0.0,0.0,0.0,0.0,1.0
19,28762550,3/24/35 23:51,3/26/35 13:35,3/24/35 23:52,3/26/35 19:47,1.83,1,1,UNKNOWN,82.227229,...,0,1,1,0,2 to 3 days,0.0,1.0,0.0,0.0,0.0
21,24474381,2/6/35 12:43,2/8/35 18:33,2/6/35 12:45,2/8/35 21:23,2.36,1,1,WHITE - RUSSIAN,86.098778,...,1,1,1,1,3 to 4 days,0.0,0.0,1.0,0.0,0.0
22,20598872,1/10/30 7:22,1/24/30 14:40,1/10/30 8:55,1/24/30 17:18,14.35,1,1,WHITE,78.025569,...,1,1,1,0,7+ days,0.0,0.0,0.0,0.0,1.0


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,diabetes,hypertension,ihd,stroke,los_icu_binned,los_icu_binned_1 to 2 days,los_icu_binned_2 to 3 days,los_icu_binned_3 to 4 days,los_icu_binned_4 to 7 days,los_icu_binned_7+ days
0,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,...,1,1,0,0,3 to 4 days,0.0,0.0,1.0,0.0,0.0
1,29179088,4/10/80 23:21,4/17/80 14:00,4/12/80 17:41,4/16/80 13:38,3.83,0,1,WHITE,72.276455,...,0,1,1,0,4 to 7 days,0.0,0.0,0.0,1.0,0.0
2,27139242,4/25/33 5:47,5/13/33 7:15,4/25/33 6:37,4/27/33 16:01,2.39,0,1,BLACK/AFRICAN AMERICAN,73.312782,...,0,0,0,0,3 to 4 days,0.0,0.0,1.0,0.0,0.0
3,25918892,10/27/43 15:06,10/31/43 14:21,10/27/43 15:06,10/30/43 12:11,2.88,0,1,WHITE,91.820358,...,0,1,1,1,3 to 4 days,0.0,0.0,1.0,0.0,0.0
4,20778724,10/4/67 7:15,10/19/67 17:45,10/5/67 9:23,10/6/67 21:43,1.51,0,1,ASIAN - CHINESE,63.756003,...,0,1,0,1,1 to 2 days,1.0,0.0,0.0,0.0,0.0


In [7]:
static_dead['los_icu_binned'].value_counts()

los_icu_binned
7+ days        476
4 to 7 days    317
1 to 2 days    249
3 to 4 days    222
2 to 3 days    186
Name: count, dtype: int64

In [8]:
static_alive['los_icu_binned'].value_counts()

los_icu_binned
7+ days        1819
3 to 4 days    1805
4 to 7 days    1752
2 to 3 days    1549
1 to 2 days    1321
Name: count, dtype: int64

In [9]:
static_dead.to_csv('../../data/static_cleaned_dead.csv', index=False)
static_alive.to_csv('../../data/static_cleaned_alive.csv', index=False)

print('saved')

saved


In [10]:
print(static_dead.shape)
print(static_alive.shape)

(1450, 30)
(8246, 30)


In [11]:
base_split_path = '../../data/split/with-outliers/partitioned'

In [12]:
static_dead_train, static_dead_test = train_test_split(
    static_dead, test_size=0.10,
    random_state=0, stratify=static_dead['los_icu_binned']
    )

static_dead_test.to_csv(f'{base_split_path}/static_dead_test.csv', index=False)

print('saved')

saved


In [13]:
static_alive_train, static_alive_test = train_test_split(
    static_alive, test_size=0.10,
    random_state=0, stratify=static_alive['los_icu_binned']
    )

static_alive_test.to_csv(f'{base_split_path}/static_alive_test.csv', index=False)

print('saved')

saved


In [14]:
static_dead_train, static_dead_val = train_test_split(
    static_dead_train, test_size=0.10,
    random_state=0, stratify=static_dead_train['los_icu_binned']
    )


static_dead_train.to_csv(f'{base_split_path}/static_dead_train.csv', index=False)
static_dead_val.to_csv(f'{base_split_path}/static_dead_val.csv', index=False)

print('saved')

saved


In [15]:
static_alive_train, static_alive_val = train_test_split(
    static_alive_train, test_size=0.10,
    random_state=0, stratify=static_alive_train['los_icu_binned']
    )


static_alive_train.to_csv(f'{base_split_path}/static_alive_train.csv', index=False)
static_alive_val.to_csv(f'{base_split_path}/static_alive_val.csv', index=False)

print('saved')

saved
